# Recommendation Systems for Speedrun.com

# Recommendation by User ID using Bipartite Graph

This method of recommendation works by finding other users that have played the same game as a target user. The games that the other users have played are ranked by occurrence. Games that the target user has not played but the other users have played are then recommended in order.

In [4]:
import pandas as pd
import csv
import networkx as nx
import numpy as np

from collections import defaultdict, Counter
from operator import itemgetter
from datetime import datetime
from tqdm import tqdm

import scipy
import random

In [53]:
user_prefs_filename = "../data/users/user_preferences_with_metadata.csv"
user_prefs_df = pd.read_csv(user_prefs_filename)
user_prefs_df = user_prefs_df[(user_prefs_df['signup_date'].notna()) & (user_prefs_df['signup_date'] != "Null")]
user_prefs_df['signup_date'] = pd.to_datetime(user_prefs_df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
user_prefs_df['signup_date'] = pd.to_datetime(user_prefs_df['signup_date'].dt.strftime('%Y-%m-%d'))
user_prefs_df = user_prefs_df[(user_prefs_df['signup_date'] < '2023-01-01')]

In [54]:
user_prefs_df.describe(include='all', datetime_is_numeric=True)

,user,signup_date,location,num_games,games
count,335322,335322,335322,335322.000000,335322
unique,335322,NaN,257,NaN,88806
top,j5wzz2qj,NaN,us,NaN,k6q4rqzd
freq,1,NaN,101439,NaN,5131
mean,NaN,2020-06-28 13:40:53.271780608,NaN,1.994465,NaN
min,NaN,2014-01-06 00:00:00,NaN,1.000000,NaN
25%,NaN,2019-09-16 00:00:00,NaN,1.000000,NaN
50%,NaN,2021-01-04 00:00:00,NaN,1.000000,NaN
75%,NaN,2021-09-25 00:00:00,NaN,2.000000,NaN
max,NaN,2022-12-31 00:00:00,NaN,2059.000000,NaN


In [55]:
exploded_games_df = user_prefs_df.copy()
exploded_games_df['games'] = exploded_games_df['games'].str.split(',')
exploded_games_df = exploded_games_df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})[['user_id', 'game_id']]

In [56]:
exploded_games_df.describe()

,user_id,game_id
count,668788,668788
unique,335322,31425
top,kj9521v8,k6q4rqzd
freq,2059,6979


In [57]:
bipartite_graph = nx.Graph()

# Users have a bipartite value of 0, games have a bipartite value of 1.
bipartite_graph.add_nodes_from(set(exploded_games_df['user_id'].values), bipartite=0)
bipartite_graph.add_nodes_from(set(exploded_games_df['game_id'].values), bipartite=1)
bipartite_graph.add_edges_from([(user, game) for user, game in zip(exploded_games_df['user_id'], exploded_games_df['game_id'])])

In [58]:
nx.is_bipartite(bipartite_graph)

True

In [59]:
del user_prefs_filename, user_prefs_df, exploded_games_df

# Overlapping Set Similarity with Limiting the Graph

There are two methods of limiting the number of user-item interactions in our bipartite graph. We can either use the mean and standard deviation of the `num_games` column, or limit based on the integer number of games played by a given user. For example, we can either use three standard deviations of the mean to have a cutoff value of `24.2 (3 s.f.)`, or we can use the value of `2` for users that have played only one game.

Using the method of standard deviations, we get a very similar output to the unlimited user-item interaction bipartite graph. We get popular games recommended most of the time. If we use the second approach, we get (anecdotally) more precise recommendations. However, the second method does not scale well, since we need to construct a different graph for each number of games played by each user. In reality, this isn't as bad as we think. Out of the 335,322 total users in our sample we can cover 306,371 users, or 91.4% (3 s.f.) of them with three graphs. 

In [60]:
def clean_user_preferences_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df[(df['signup_date'].notna()) & (df['signup_date'] != "Null")]
    df['signup_date'] = pd.to_datetime(df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
    df['signup_date'] = pd.to_datetime(df['signup_date'].dt.strftime('%Y-%m-%d'))
    df = df[(df['signup_date'] < '2023-01-01')]
    return df

def limit_number_games_user_preferences_df(df: pd.DataFrame, num_games: int) -> pd.DataFrame:
    return df[(df['num_games'] <= num_games)]

def explode_games_played(df: pd.DataFrame) -> pd.DataFrame:
    df['games'] = df['games'].str.split(',')
    return df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})

def recommendation_graph_for_n_games_played(df: pd.DataFrame, n: int) -> tuple[pd.DataFrame, nx.Graph]:
    df = clean_user_preferences_df(df)
    df = limit_number_games_user_preferences_df(df, n+1)
    df = explode_games_played(df)
    bipartite_graph = nx.Graph()
    bipartite_graph.add_nodes_from(set(df['user_id'].values), bipartite=0)
    bipartite_graph.add_nodes_from(set(df['game_id'].values), bipartite=1)
    bipartite_graph.add_edges_from([(user, game) for user, game in zip(df['user_id'], df['game_id'])])
    return df, bipartite_graph

In [61]:
user_prefs_df = pd.read_csv('../data/users/user_preferences_with_metadata.csv')
user_prefs_df, bipartite_graph = recommendation_graph_for_n_games_played(user_prefs_df, 5)

/var/folders/fp/pwtrzcy130zdfcx62mp9_kmc0000gn/T/ipykernel_2763/4036755379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['signup_date'] = pd.to_datetime(df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
/var/folders/fp/pwtrzcy130zdfcx62mp9_kmc0000gn/T/ipykernel_2763/4036755379.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['signup_date'] = pd.to_datetime(df['signup_date'].dt.strftime('%Y-%m-%d'))


In [62]:
def user_similarity(bipartite_graph: nx.Graph, total_item_nodes: int, user_a_id: str, user_b_id: str) -> float:
    assert bipartite_graph.nodes[user_a_id]['bipartite'] == 0
    assert bipartite_graph.nodes[user_b_id]['bipartite'] == 0

    a_neighbours = bipartite_graph.neighbors(user_a_id)
    b_neighbours = bipartite_graph.neighbors(user_b_id)
    shared_nodes = set(a_neighbours).intersection(b_neighbours)

    return len(shared_nodes) / total_item_nodes

def most_similar_users(bipartite_graph: nx.Graph, user_id: str) -> tuple[list[str], float]:
    all_users = set([user for user, value in bipartite_graph.nodes(data=True) if value['bipartite'] == 0])
    all_users.remove(user_id)

    total_item_nodes = 0
    for _, values in bipartite_graph.nodes(data=True):
        if values['bipartite'] == 1: total_item_nodes += 1

    similarities = defaultdict(float)
    for user in all_users:
        similarities[user] = user_similarity(bipartite_graph, total_item_nodes, user_id, user)

    max_similarity = max(similarities.values())
    return [user for user, similarity in similarities.items() if similarity == max_similarity], max_similarity

def recommend_games(bipartite_graph: nx.Graph, user_id: str) -> list[str]:
    similar_users, _ = most_similar_users(bipartite_graph, user_id)
    other_games = [game for user in similar_users for game in bipartite_graph.neighbors(user)]
    game_rankings = Counter(other_games)

    already_played_games = set(bipartite_graph.neighbors(user_id))

    try:
        [game_rankings.pop(game) for game in already_played_games]
    except KeyError:
        # If no other users in data set have played this game.
        pass

    ranked_games_in_order, _ = list(zip(*sorted(game_rankings.items(), key=itemgetter(1), reverse=True)))
    
    return ranked_games_in_order

In [63]:
games_metadata_df = pd.read_csv('../data/games/metadata/all_games.csv').rename(columns={'game_id': 'id'})

In [64]:
user = "x355n6qj"

played_games = list(bipartite_graph.neighbors(user))
print(f"Played games: {games_metadata_df[(games_metadata_df['id'].isin(played_games))].game_name.values}")

recommended_games = recommend_games(bipartite_graph, user)
print(f"Recommended games: {games_metadata_df.set_index('id').loc(axis=0)[recommended_games].game_name.values[:20]}")

Played games: ['Hello Neighbor' 'Super Mario Odyssey']
Recommended games: ["Baldi's Basics Category Extensions" 'Google Quick Draw'
 'Snipperclips: Cut it out  together!' 'Hello Neighbor 2' 'Cuphead'
 'Super Mario Sunshine' 'Marble Saga: Kororinpa'
 'The Legend of Zelda: The Wind Waker HD' 'Clicker Heroes'
 'Minecraft: Java Edition' 'Island Saver'
 'Super Mario Odyssey Category Extensions']


In [65]:
del games_metadata_df, played_games, recommended_games, user_prefs_df, bipartite_graph, user

# Recommendation Using a Game Similarity Matrix

This recommendation method works by creating a matrix of how users have rated different games. We construct this matrix by rating a game `1` if a user has played it, and `0` if not. We then normalise these values by making the sum of ratings by each user equal to `1`. This is also called [scaling to a unit length](https://en.wikipedia.org/wiki/Feature_scaling#Scaling_to_unit_length). We take the dot product of the matrix and the transposed matrix, and this gives us the similarity between each item in the data set. The method is taken from [here](https://towardsdatascience.com/recommender-systems-item-customer-collaborative-filtering-ff0c8f41ae8a). **This method does not scale very well**.

In [31]:
user_prefs_df = pd.read_csv('../data/users/user_preferences_with_metadata.csv')

In [36]:
def format_user_prefs_df_to_ratings(df: pd.DataFrame, number_users=-1) -> pd.DataFrame:
    tmp_df = df.copy()[:number_users]
    tmp_df['games'] = tmp_df['games'].str.split(',')
    tmp_df = tmp_df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})
    tmp_df['rating'] = 1
    return tmp_df[['user_id', 'game_id', 'rating']]

def construct_similarity_matrix(df: pd.DataFrame) -> pd.DataFrame:
    tmp_df = df.copy()
    tmp_df = pd.pivot_table(tmp_df, values='rating', index='user_id', columns='game_id')
    tmp_df = tmp_df.fillna(0.0)
    normalised_tmp_df = tmp_df / np.sqrt(np.square(tmp_df).sum(axis=0))
    return normalised_tmp_df.transpose().dot(normalised_tmp_df)

def recommend_from_game(similarity_matrix: pd.DataFrame, game_id: str, n_recommendations: int) -> list[str]:
    return similarity_matrix.nlargest(n_recommendations+1, game_id).index.values[1:]

In [33]:
similarity_matrix = pd.DataFrame()
generate = False

if generate:
    ratings_df = format_user_prefs_df_to_ratings(user_prefs_df, number_users=5000)
    similarity_matrix = construct_similarity_matrix(ratings_df)
    similarity_matrix.to_csv('./test.csv')
else:
    similarity_matrix = pd.read_csv('../data/users/game_similarity_matrix.csv')

In [34]:
similarity_matrix = similarity_matrix.set_index('game_id')

In [35]:
games_metadata_df = pd.read_csv('../data/games/metadata/all_games.csv').rename(columns={'game_id': 'id'})

In [37]:
game_id = "kdkmzmx1"
n = 5
recommended_games = recommend_from_game(similarity_matrix, game_id, n)

print(f"Played Game: {games_metadata_df.set_index('id').loc(axis=0)[game_id].game_name}")
print(f"Recommended games: {games_metadata_df.set_index('id').loc(axis=0)[recommended_games].game_name.values}")

Played Game: LEGO Harry Potter: Years 5-7 (DS/PSP)
Recommended games: ['LEGO Indiana Jones 2: The Adventure Continues (DS)'
 'LEGO Star Wars II: The Original Trilogy (PSP)' 'Escape PS1 Hagrid'
 'LEGO Star Wars III: The Clone Wars (DS)'
 'LEGO Pirates of the Caribbean: The Video Game (DS)']


In [8]:
del game_id, generate, recommended_games, user_prefs_df, games_metadata_df

# Metrics of Cosine Similarity CF

In [53]:
from sklearn.model_selection import train_test_split

In [122]:
def format_user_prefs_df_to_ratings(df: pd.DataFrame, number_users=-1) -> pd.DataFrame:
    tmp_df = df.copy()[:number_users]
    tmp_df['games'] = tmp_df['games'].str.split(',')
    tmp_df = tmp_df.explode('games').rename(columns = {'games': 'game_id', 'user':'user_id'})
    tmp_df['rating'] = 1
    return tmp_df[['user_id', 'game_id', 'rating']]

def construct_similarity_matrix(df: pd.DataFrame) -> pd.DataFrame:
    tmp_df = df.copy()
    tmp_df = pd.pivot_table(tmp_df, values='rating', index='user_id', columns='game_id')
    tmp_df = tmp_df.fillna(0.0)
    normalised_tmp_df = tmp_df / np.sqrt(np.square(tmp_df).sum(axis=0))
    return normalised_tmp_df.transpose().dot(normalised_tmp_df)

In [162]:
user_prefs_df = pd.read_csv('../data/users/user_preferences_with_metadata.csv').sort_values(by='num_games', ascending=True)[:2000]

In [163]:
train_df, test_df = train_test_split(user_prefs_df, test_size=0.2, random_state=0)

# Create cosine similarity matrix for test data, create recommendation engine using that cosine similarity matrix. 
train_df = format_user_prefs_df_to_ratings(train_df)
similarity_matrix_t = construct_similarity_matrix(train_df)

# Find the games played by the test users. Only use them if they play more than one game.
recommendations = []
unique_games_in_test = np.unique(test_df['games'].str.split(',').tolist()[0])
for game in unique_games_in_test:
    game_row = similarity_matrix_t.loc[game]
    recs = game_row[(game_row > 0.5).any() & (game_row != 1.0)].index.values.tolist()[0]
    recommendations.append(recs)

relevant_games_set = [set(games) for games in unique_games_in_test]
recommended_games_set = [set(games) for games in recommendations]
intersection = [len(recommended.intersection(relevant)) for recommended, relevant in zip(recommended_games_set, relevant_games_set)]
recall = np.mean([i/len(relevant) for i, relevant in zip(intersection, relevant_games_set)])
precision = np.mean([i/len(recommended) for i, recommended in zip(intersection, recommended_games_set)])
f1_score = 2 * (precision * recall) / (precision + recall)

In [164]:
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1-score: {f1_score}")

Recall: 0.5
Precision: 0.6666666666666666
F1-score: 0.5714285714285715


# Recommendation using Node2Vec Embeddings

The idea behind using node2vec embeddings for recommendation is to predict future links that don't already exist. We can prove that this works for individual games recommendation by removing selected edges and using cosine similarity of embeddings to predict which edges should exist given this graph. We carry this on further by creating a pipeline to predict games to play when they are completely disconnected.

Do [this](https://sparsh-ai.github.io/rec-tutorials/graph%20embedding%20movielens%20factorization/2021/04/24/Recommendation-Node2vec.html)?

In [204]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, auc, roc_curve, roc_auc_score,confusion_matrix

In [205]:
edge_existence_df = pd.read_csv('../data/games/network/edge_existence_dataset.csv').set_index('index')
embeddings_df = pd.read_csv('../data/games/network/homophily.emb', delimiter=" ", skiprows=1, index_col=0, header=None).rename(columns={0: 'id'}).add_prefix("dim_")

In [206]:
edge_features = [(embeddings_df.loc[node1].values.tolist()+embeddings_df.loc[node2].values.tolist()) for node1, node2 in zip(edge_existence_df["source"], edge_existence_df["target"])]

In [207]:
X = np.array(edge_features)
y = edge_existence_df['connection']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [208]:
X_train.shape, y_train.shape

((10471, 32), (10471,))

In [209]:
clf1 = RandomForestClassifier()
 
# parameters
param = {'n_estimators' : [10,50,100], 'max_depth' : [5,10,15]}
 
# model
grid_clf_acc1 = GridSearchCV(clf1, param_grid = param)
 
# train the model
grid_clf_acc1.fit(X_train, y_train)
 
print('Grid best parameter (max. accuracy): ', grid_clf_acc1.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc1.best_score_)
 
# alternative metric to optimize over grid parameters: AUC
grid_clf_auc1 = GridSearchCV(clf1, param_grid = param, scoring = 'roc_auc')
grid_clf_auc1.fit(X_train, y_train)
predict_proba = grid_clf_auc1.predict_proba(X_test)[:,1]
 
print('Test set AUC: ', roc_auc_score(y_test, predict_proba))
print('Grid best parameter (max. AUC): ', grid_clf_auc1.best_params_)
print('Grid best score (AUC): ', grid_clf_auc1.best_score_)

Grid best parameter (max. accuracy):  {'max_depth': 15, 'n_estimators': 50}
Grid best score (accuracy):  0.9985675631933949
Test set AUC:  0.999998330779426
Grid best parameter (max. AUC):  {'max_depth': 10, 'n_estimators': 50}
Grid best score (AUC):  0.9999765448696089


In [214]:
confusion_matrix(grid_clf_auc1.predict(X_test),y_test)

array([[ 929,    0],
       [  10, 2552]], dtype=int64)

In [215]:
# Test if there is a link between super mario odyssey and super mario 64 (there is).
# IDs: 76r55vd8 o1y9wo6q
SMO = [0.003971752, 0.00646716, 0.005253054, 0.000770621, 0.023088027, -0.0025586337, 0.021919776, 0.023069408, 0.00055524334, 0.018905181, -0.015266668, -0.0029585548, 0.010636453, -0.017556641, -0.0020937473, -0.029378641, 0.013062697, -0.009451449, -0.020212192, 0.017371856, 0.003602054, -0.026192848, -0.025948, 0.017900482, 0.02082228, -0.019447599, 0.020568952, 0.025263663, -0.01999519, -0.023548912, -0.019287951, -0.011146348]
SM64 = [-0.012588676, -0.029154047, -0.0028756857, -0.009487752, 0.017457027, 0.0210394, -0.010649294, -0.0018646903, 0.022166133, -0.024311021, 0.0062766, -0.029152796, -0.030749485, 0.0017411523, 0.029207725, -0.015724432, 0.009961858, 0.008669887, 0.01067755, -0.012766957, 0.0050436445, -0.0035253912, -0.008545488, -0.022413097, -0.014777329, -0.005886156, -0.024985388, 0.004289292, -0.0025947131, 0.0019044504, -0.014002968, -0.02486419]

smo_sm64_features = np.array(SMO + SM64).reshape(1, -1)
grid_clf_auc1.predict(smo_sm64_features)

ValueError: X has 64 features, but RandomForestClassifier is expecting 32 features as input.

# Uhoh

In [202]:
# classifier
clf3 = MLPClassifier(max_iter=1000)
 
# scaling training and test sets
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
 
# parameters
param = {'hidden_layer_sizes' : [10,100,[10,10]], 'activation' : ['tanh','relu'], 'solver' : ['adam','lbfgs']}
 
# model
grid_clf_acc3 = GridSearchCV(clf3, param_grid = param)
 
# train the model
grid_clf_acc3.fit(X_train_scaled, y_train)
 
print('Grid best parameter (max. accuracy): ', grid_clf_acc3.best_params_)
print('Grid best score (accuracy): ', grid_clf_acc3.best_score_)
 
# alternative metric to optimize over grid parameters: AUC
grid_clf_auc3 = GridSearchCV(clf3, param_grid = param, scoring = 'roc_auc')
grid_clf_auc3.fit(X_train_scaled, y_train)
predict_proba = grid_clf_auc3.predict_proba(X_test_scaled)[:,1]
 
print('Test set AUC: ', roc_auc_score(y_test, predict_proba))
print('Grid best parameter (max. AUC): ', grid_clf_auc3.best_params_)
print('Grid best score (AUC): ', grid_clf_auc3.best_score_)

C:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the nu

KeyboardInterrupt: 